In [1]:
import yfinance as yf # For loading financial datasets
import pandas as pd # For creating tables and dataframes
import math 
import numpy as np

stocks = ["AAPL", "AMZN", "MSFT", "NFLX", "GOOG", "TSLA", "GME"] # 7 stocks needed for the first table

In [2]:
weight = 1 / 7 # This will be the portfolio weight for each ticker 
portfolio_weight = [weight]*7

In [3]:
def get_annualized_volatility(dataframe):
    aapl_resampled = dataframe.resample('3M').asfreq() #Resample data from daily to every 3-months
    dataframe["pct_change"] = dataframe["Adj Close"].pct_change() #Calculate returns for every 3-months
    dataframe.dropna(inplace=True) #Drop any NaN values (Must drop!)
    annualized_volatility = dataframe["pct_change"].std() * math.sqrt(4)
    return annualized_volatility

In [4]:
def beta_against_market(ticker, market):
    dataframe = yf.download(f"{ticker} {market}")["Adj Close"]
    dataframe.dropna(inplace=True)
    dataframe_resampled = dataframe.resample("12M").sum() # Resample data to use trailing 12-months
    covariance = dataframe_resampled.cov().iloc[0, 1] # left of the comma is for row selection [row, column]
    beta = covariance / dataframe_resampled[market].var()
    return beta

In [9]:
def get_weekly_drawdowns(dataframe):
    weekly_drawdowns = []
    weekly_resample = dataframe.resample("5D").asfreq()["Adj Close"] #resample for weekly data (5 trading days per week)
    weekly_resample.dropna(inplace=True)
    weekly_resample_array = np.array(weekly_resample)
    for i in range(0, len(weekly_resample_array)-52, 52): # Breakdown dataset into 52 weeks to find max and minimum during the 52-week period
        high = np.max(weekly_resample_array[i:i+52])
        low = np.min(weekly_resample_array[i:i+52])
        calc = (low - high) / high
        weekly_drawdowns.append(calc)
    average_weekly_drawdowns = sum(weekly_drawdowns) / len(weekly_drawdowns)
    maximum_weekly_drawdown = max(weekly_drawdowns)
    return average_weekly_drawdowns, maximum_weekly_drawdown

In [10]:
def get_total_return(dataframe):
    #.iloc[-1] get last row and .iloc[0] get first row of Adj Close 
    #Formula for Total_Return = New_Stock_price - Current_Stock_Price / Current_Stock_Price
    total_return = (dataframe["Adj Close"].iloc[-1] - dataframe["Adj Close"].iloc[0]) / (dataframe["Adj Close"].iloc[0])
    return f"{total_return*100}%"

In [11]:
def get_total_annualized_return(dataframe):
    resampled_data = dataframe.resample("10Y").asfreq()["Adj Close"]
    resampled_data.dropna(inplace=True)
    total_annualized_return = (resampled_data.iloc[-1] - resampled_data.iloc[0]) / resampled_data.iloc[0]
    return f"{total_annualized_return * 100}%"

In [12]:
#Create table
tickers = []
annualized_volatility = []
beta_spy = []
beta_iwm = []
beta_dia = []
average_weekly_drawdown = []
maximum_weekly_drawdown = []
total_return = []
total_annualized_return = []
for stock in stocks: # This will loop through the 7 stocks to get the data needed
    get_dataframe = yf.download(stock)
    tickers.append(stock)
    annualized_volatility.append(get_annualized_volatility(get_dataframe))
    beta_spy.append(beta_against_market(stock, "SPY"))
    beta_iwm.append(beta_against_market(stock, "IWM"))
    beta_dia.append(beta_against_market(stock, "DIA"))
    average_weekly_drawdown.append(get_weekly_drawdowns(get_dataframe)[0])
    maximum_weekly_drawdown.append(get_weekly_drawdowns(get_dataframe)[1])
    total_return.append(get_total_return(get_dataframe))
    total_annualized_return.append(get_total_annualized_return(get_dataframe))

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%********

In [21]:
create_table = {"ticker" : tickers, "portfolio weight": portfolio_weight, 
                "annualized volatility" : annualized_volatility, "beta spy" : beta_spy,
               "beta iwm": beta_iwm, "beta dia": beta_dia, "average weekly drawdown": average_weekly_drawdown,
               "maximum weekly drawdown": maximum_weekly_drawdown, "total return" : total_return, 
               "total annualized return" : total_annualized_return}
create_dataframe = pd.DataFrame(create_table)

In [24]:
table_one = create_dataframe
table_one

,ticker,portfolio weight,annualized volatility,beta spy,beta iwm,beta dia,average weekly drawdown,maximum weekly drawdown,total return,total annualized return
0,AAPL,0.142857,0.056498,0.345282,0.706205,0.432075,-0.460454,-0.211499,155117.40849120982%,110503.25688874073%
1,AMZN,0.142857,0.072302,0.441228,0.854588,0.504999,-0.492303,-0.204375,108044.99830923228%,1745.1095093625365%
2,MSFT,0.142857,0.042667,0.631716,1.275094,0.778177,-0.342534,-0.131707,396603.0853894255%,6064.7560973697355%
3,NFLX,0.142857,0.071829,1.426548,2.847636,1.759929,-0.554317,-0.282087,26110.74247089914%,1581.9246861227023%
4,GOOG,0.142857,0.038903,0.311484,0.614288,0.379117,-0.333796,-0.159350,3212.1555128461887%,446.6324740288087%
5,TSLA,0.142857,0.072339,0.639261,1.202562,0.750205,-0.525999,-0.267987,12292.779640578823%,13149.53242129483%
6,GME,0.142857,0.101358,0.064708,0.125535,0.074969,-0.487723,-0.249868,1060.8494711732742%,431.3860768298257%


In [67]:
portfolio_rets_df.columns = ["portfolio_rets", "daily_spy_rets"]

In [69]:
portfolio_rets_df["P - B"] = portfolio_rets_df["portfolio_rets"] - portfolio_rets_df["daily_spy_rets"]

In [72]:
tracking_error = portfolio_rets_df["P - B"].std()

In [73]:
#Prepare annual data for sharpe ratio
get_stock_data = yf.download(stocks)["Adj Close"]
get_stock_data.dropna(inplace=True)
data_resampled = get_stock_data.resample("Y").asfreq()
data_resampled.dropna(inplace=True)


annual_pct_change = data_resampled.pct_change()[1:] 
weighted_annual_rets = annual_pct_change * portfolio_weight
portfolio_annual_rets = weighted_annual_rets.sum(axis=1)

#Compute Sharpe Ratio
expected_portfolio_return = portfolio_annual_rets.mean()
portfolio_std = portfolio_annual_rets.std()
risk_free_rate = 0.0471 # 3-Months T-bill current interest


sharpe_ratio = (expected_portfolio_return - risk_free_rate) / portfolio_std

[*********************100%***********************]  7 of 7 completed


In [102]:
#Annualized Volatility spread (Portfilio Volatility - ETF Volatility)
dataframe = yf.download(stocks)["Adj Close"]
dataframe.dropna(inplace=True)
dataframe_resampled = dataframe.resample("252D").asfreq() # Resample every 252 days
dataframe_resampled.dropna(inplace=True)

[*********************100%***********************]  7 of 7 completed


In [104]:
dataframe_pct_change = dataframe_resampled.pct_change()[1:]

In [106]:
portfolio_ret_252d = dataframe_pct_change * portfolio_weight
final_return_252d = portfolio_ret_252d.sum(axis=1)

In [108]:
final_return_252d.std()

0.583155764724233

In [110]:
etf_volatility = yf.download("SPY")["Adj Close"]
resample_etf_252 = etf_volatility.resample("252D").asfreq()
resample_etf_252.pct_change()[1:].std()


[*********************100%***********************]  1 of 1 completed


0.13048281657796715

In [147]:
def get_corr_against_etf(data):
    return data.corr().iloc[0, 1]

def get_cov_against_etf(data):
    return data.corr().iloc[0, 1]

def get_tracking_error(data):
    data["P - B"] = data["portfolio_rets"] - data["daily_etf_rets"]
    return data["P - B"].std()

def get_volatility_spread(stock_data, etf):
    stock_data_resampled = stock_data.resample("252D").asfreq()
    stock_data_resampled.dropna(inplace=True)
    stock_data_pct = stock_data_resampled.pct_change()[1:]
    portfolio_ret_252d = stock_data_pct * portfolio_weight
    final_return_252d = portfolio_ret_252d.sum(axis=1)
    portfolio_volatility = final_return_252d.std()
    get_etf = yf.download(etf)["Adj Close"].pct_change()[1:]
    etf_volatility = get_etf.std()
    return portfolio_volatility - etf_volatility

In [148]:
etf_tickers = ["SPY", "IWM", "DIA"]

correlations = []
covariances = []
sharpe_ratios = [sharpe_ratio]*3
tracking_errors = []
volatility_spreads = []


for ticker in etf_tickers:
    # To get correlation of portfolio against ETF we first need to get daily returns of portfolio
    stock_data = yf.download(stocks)["Adj Close"]
    stock_data.dropna(inplace=True)
    
    weighted_rets = (portfolio_weight * stock_data) #stock * weight = portfolio_return accross all 7 stocks 
    portfolio_rets = weighted_rets.sum(axis=1) #https://www.codingfinance.com/post/2018-04-05-portfolio-returns-py/
    
    
    etf = yf.download(ticker, start="2010-06-29")["Adj Close"]
    etf_pct_change = etf.pct_change()[1:]
    portfolio_rets_df = pd.DataFrame(portfolio_rets) #convert type series to dataframe
    portfolio_rets_df["daily_etf_rets"] = etf_pct_change
    portfolio_rets_df.columns = ["portfolio_rets", "daily_etf_rets"]
    correlations.append(get_corr_against_etf(portfolio_rets_df))
    covariances.append(get_cov_against_etf(portfolio_rets_df))
    tracking_errors.append(get_tracking_error(portfolio_rets_df))
    volatility_spreads.append(get_volatility_spread(stock_data, ticker))

[*********************100%***********************]  7 of 7 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  7 of 7 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  7 of 7 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [152]:
table_two = {"tickers": etf_tickers, "correlation" : correlations, "covariances": covariances, 
             "sharpe_ratio": sharpe_ratios, "tracking error": tracking_errors, 
             "annualized_volatility_spread": volatility_spreads}
table_two = pd.DataFrame(table_two)

In [153]:
table_two

,tickers,correlation,covariances,sharpe_ratio,tracking error,annualized_volatility_spread
0,SPY,0.003248,0.003248,1.107072,69.326409,0.571238
1,IWM,-0.001227,-0.001227,1.107072,69.326462,0.567863
2,DIA,0.000328,0.000328,1.107072,69.326441,0.571111
